In [1]:
from goose import Goose
import unicodedata
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import re


print("Imports Realizados!")

Imports Realizados!


In [2]:
def remover_naoLatin(string): 
    new_chars = []
    for char in string:
        if char == '\n':
            new_chars.append(' ')
            continue
        try:
            if unicodedata.name(unicode(char)).startswith(('LATIN', 'SPACE')):
                new_chars.append(char)
        except:
            continue
    return ''.join(new_chars)

In [3]:
def normalizar_palavra(palavra):

    # Unicode normalize transforma um caracter em seu equivalente em latin.
    palavraSemAcento = palavra.lower()
    try:
        nfkd = unicodedata.normalize('NFKD', palavra)
        palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
        palavraSemAcento = palavraSemAcento.lower()
    except:
        pass
    
    # Usa expressão regular para retornar a palavra apenas com números, letras e espaço
    return re.sub('[^a-zA-Z0-9 \\\]', '', palavraSemAcento)

In [4]:
def remover_stopwords(texto):
    c = []
    for i in stopwords.words("portuguese"):
        if len(i) > 1:
            c.append(i)
    saida = ""
    for palavra in c:
        saida = saida + " " + palavra

    x = normalizar_palavra(saida)  
    #print(x + "\n")

    p = ""
    for _p in texto.split():
        if _p not in x.split():
            if len(_p) > 1:
                p = p + _p + " "
    return p

In [5]:
def processar_texto(texto):
    texto = remover_naoLatin(texto)
    texto = normalizar_palavra(texto)
    return remover_stopwords(texto)
    

In [35]:
data = pd.read_csv('out.csv')
g = Goose()

categorias = data['CATEGORIA']
urls = data['URL']

file = open("saida_maior.txt", "w")
for url, categoria in zip(urls, categorias):
    artigo = g.extract(url=url)
    #texto = artigo.meta_description
    texto = artigo.cleaned_text
    lista_palavras = processar_texto(texto)
    if(len(lista_palavras) > 1):
        file.write(categoria + "," + lista_palavras + '\n')

file.close()

In [36]:
df = pd.read_table('./saida_maior.txt',sep=',',header=None, names=['CATEGORIA','TEXTO'])

#df.describe()
df.head()

,CATEGORIA,TEXTO
0,noticia,rosinha deixou cadeia publica benfica zona nor...
1,noticia,brasilia policia federal apreendeu gabinete ae...
2,noticia,superintendencia regional policia federal pf r...
3,noticia,policias federal militar homens forcas armadas...
4,noticia,termina nesta quintafeira feirao limpa nome se...


In [37]:
#transformar noticia em 1 e esporte em 0
encoded_labels = df['CATEGORIA'].map(lambda x: 1 if x == 'noticia' else 0).values


X_train, X_test, y_train, y_test = train_test_split(df['TEXTO'], encoded_labels)

count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)


In [38]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

predictions = naive_bayes.predict(testing_data)
print(predictions)

[1 0 1 0 1 0 1 0 0 1]


In [39]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print('Accuracy score: {0:.1f}%.'.format(accuracy * 100))

Accuracy score: 100.0%.


In [40]:
def get_new_test(url):
    g = Goose()
    artigo = g.extract(url=url)
    texto = artigo.meta_description
    lista_palavras = processar_texto(texto)
    return lista_palavras

In [43]:
url = "https://globoesporte.globo.com/futebol/noticia/bandeira-branca-apos-polemica-thiago-neves-diz-querer-jogar-ao-lado-de-luan.ghtml"
noticia = get_new_test(url)
noticia_transformada = count_vector.transform([noticia])
pred = naive_bayes.predict(noticia_transformada)

print("noticia" if pred == 1 else "esporte")

esporte
